# 完全なビジネスソリューション
「会社のWebサイトから情報を収集し、LLM（大規模言語モデル）を活用して会社パンフレット（Brochure）を自動生成する」ビジネスソリューションを構築する内容。

## 今、私たちは1日目から次のレベルにプロジェクトを取ります

### ビジネスチャレンジ：

将来のクライアント、投資家、潜在的な新兵に使用される企業のパンフレットを構築する製品を作成します。

会社名とその主要なウェブサイトが提供されます。

実際のビジネスアプリケーションの例については、このノートブックの終わりを参照してください。

そして覚えておいてください：あなたが問題やアイデアがあるならば、私はいつでも利用可能です！手を差し伸べてください。

In [1]:
# import
# これらが失敗した場合は、適切に構築された仮想環境環境から実行していることを確認してください

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# 初期化と定数

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("APIキーはこれまでのところ良く見えます")
else:
    print("あなたのAPIキーに問題があるかもしれませんか？トラブルシューティングノートブックにアクセスしてください！")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

APIキーはこれまでのところ良く見えます


Webサイトをスクレイピングし、タイトル・本文テキスト・リンク一覧を取得するためのWebsiteクラスを実装

In [3]:
# ウェブページを表すクラス

# 一部のWebサイトでは、適切なヘッダーを使用するときに使用する必要があります。
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    スクレイピングしたウェブサイトを表すユーティリティクラス。リンクも追加されている。
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

例：https://edwarddonner.com の リンク一覧

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## 最初のステップ：GPT-4o-miniにどのリンクが関連しているかを把握してもらう

### GPT-4o-miniへの呼び出しで、Webページのリンクを読み取り、構造化されたJSONで応答
どのリンクが関連するかを決定し、"/about"などの相対的なリンクを"https://company.com/about"に置き換える必要があります。  
プロンプトでどのように応答するかの例を提供する"one shot prompting"を使用します。

これは、微妙な理解を必要とするため、LLMにとって優れたユースケースです。 Webページを解析して分析して、LLMSなしでこれをコーディングしようとすることを想像してください - それは非常に難しいでしょう！

サイドノート：「構造化された出力」と呼ばれるより高度な手法があり、スペックに従って応答するためにモデルが必要です。私たちは、自律剤AIプロジェクト中に8週目にこの手法をカバーしています。

In [5]:
link_system_prompt = "ウェブページ上のリンクのリストが表示されます。\
会社概要ページ、会社概要ページ、採用情報ページなど、\
会社案内パンフレットに掲載するのに最適なリンクを選択できます。\n"
link_system_prompt += "次の例のように JSON で応答する必要があります。"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

ウェブページ上のリンクのリストが表示されます。会社概要ページ、会社概要ページ、採用情報ページなど、会社案内パンフレットに掲載するのに最適なリンクを選択できます。
次の例のように JSON で応答する必要があります。
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



LLMを使ってリンク一覧から「パンフレットにふさわしい」リンク（例：会社概要、採用情報など）を選ぶ

get_links = Website(スクレイピング) + get_links_user_prompt(プロンプト生成) + LLM

In [7]:
def get_links_user_prompt(website):
    user_prompt = f"以下はウェブサイトのリンク一覧です {website.url} - "
    user_prompt += "これらのうち、会社案内のパンフレットに関連するWebリンクを決定し、JSON形式の完全なhttps URLを返信してください。\
    利用規約、プライバシー、メールリンクは含めないでください。\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

以下はウェブサイトのリンク一覧です https://edwarddonner.com - これらのうち、会社案内のパンフレットに関連するWebリンクを決定し、JSON形式の完全なhttps URLを返信してください。    利用規約、プライバシー、メールリンクは含めないでください。
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https:

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# 人類は彼らのサイトを削るのを難しくしているので、私はHugging Faceを使用しています。

# 「多くのサイトがスクレイピング対策（JavaScript生成、Bot検知、CAPTCHAなど）をしていて面倒だが、
# Hugging Face は比較的簡単にスクレイピングできるので使っている」という意味。

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'inference/get-started',
 '/spaces',
 '/models',
 '/moonshotai/Kimi-K2-Instruct',
 '/Qwen/Qwen3-235B-A22B-Instruct-2507',
 '/Qwen/Qwen3-Coder-480B-A35B-Instruct',
 '/mistralai/Voxtral-Mini-3B-2507',
 '/mistralai/Voxtral-Small-24B-2507',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/akhaliq/anycoder',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/NihalGazi/Text-To-Speech-Unlimited',
 '/spaces',
 '/datasets/common-pile/caselaw_access_project',
 '/datasets/NousResearch/Hermes-3-Dataset',
 '/datasets/microsoft/rStar-Coder',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/snorkelai/agent-finance-reasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## 2番目のステップ：パンフレットを作りましょう！
さらに抽出したリンク先ページもスクレイピングし、会社トップページと情報をまとめて、LLMに渡し「会社パンフレットの本文（Markdown形式）」を生成

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("発見されたリンク：", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

発見されたリンク： {'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on the Hub 🔥
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
moonshotai/Kimi-K2-Instruct
Updated
about 22 hours ago
•
195k
•
1.74k
Qwen/Qwen3-235B-A22B-Instruct-2507
Updated
1 day ago
•
1.32k
•
391
Qwen/Qwen3-Coder-480B-A35B-Instruct
Updated
about 2 hours ago
•
323
mistralai/Voxtral-Mini-3B-2507
Updated
5 days ago
•
46.4k
•
413
mistralai/Voxtral-Small-24B-2507
Updated
5 days ago
•
1.9k
•
33

In [14]:
system_prompt = "あなたは、企業のウェブサイトから関連ページをいくつか分析し、\
見込み顧客、投資家、そして採用担当者向けに会社概要の短いパンフレットを作成するアシスタントです。\
マークダウン形式で回答してください。企業文化、顧客、そしてキャリア／職種に関する情報があれば、詳細も含めてください。"

get_brochure_user_prompt = get_all_details + user_prompt

In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"あなたがご覧になっているのは、次の会社です: {company_name}\n"
    user_prompt += f"ここにランディング ページとその他の関連ページの内容を示します。この情報を使用して、Markdown で会社の短いパンフレットを作成します。\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # 5,000文字以上の場合は切り捨てます
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

発見されたリンク： {'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'あなたがご覧になっているのは、次の会社です: HuggingFace\nここにランディング ページとその他の関連ページの内容を示します。この情報を使用して、Markdown で会社の短いパンフレットを作成します。\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGet started with Inference in seconds 🚀\nReachy Mini: The Open Robot for AI Builders\nWelcome Cohere on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2-Instruct\nUpdated\nabout 22 hours ago\n•\n195k\n•\n1.74k\nQwen/Qwen3-235B-A22B-Instruct-2507\nUpdated\n1 day ago\n•\n1.32k\n•\n391\nQwen/Qwen3-Coder-480B-A35B-Instruct\nUpdated\nabout 2 hours ago\n•\n323\nmistralai/Voxtral-Mini-3B-2507\nUpdated\n5 days ago\n•\n46.4k\n•\n413\nmistralai/Voxtral-Small-24B-2507\nUpdated\n5 days ago\n•\n1.9k\

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

発見されたリンク： {'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


```markdown
# Hugging Face Company Overview

## Introduction
Hugging Face is a leading collaboration platform designed for the machine learning community, where enthusiasts, engineers, and researchers can come together to share, explore, and experiment with open-source machine learning (ML). Our mission is to empower the development of ethical AI through community collaboration and innovation.

## Company Culture
At Hugging Face, we prioritize collaboration, inclusivity, and open-source principles. We believe in fostering a supportive environment where knowledge sharing accelerates personal and professional growth. Our community-driven approach encourages participation, making it an ideal place for anyone passionate about AI.

## Key Offerings
- **Models**: Access a vast repository of over 1 million pre-trained models, enabling users to find the right solution for their specific needs.
- **Datasets**: A diverse collection of over 250,000 datasets for various ML tasks, designed to help users train and test their models effectively.
- **Spaces**: An interactive environment where users can run and showcase AI applications, fostering creativity and innovation.
- **Enterprise Solutions**: Customized solutions tailored for organizations, featuring advanced security and support, starting at $20/user/month. 

## Community Engagement
Hugging Face is proud of its robust community of more than 50,000 organizations, including industry leaders like Meta, Google, and Microsoft. We actively promote community events and provide resources for users to build their ML portfolios through shared projects.

## Career Opportunities
Hugging Face is constantly seeking talented individuals to join our diverse team. We offer various roles that cater to different skill sets, from machine learning engineers to community managers. Our work environment is flexible and supportive, providing employees with opportunities to directly impact the AI landscape.

## Vision for the Future
We are committed to leading the way in ethical AI development and fostering an environment where innovation thrives. Our goal is to cultivate a rich ecosystem that supports the growth of both individuals and the overall AI community.

---

For more information, visit our website: [Hugging Face](https://huggingface.co)
```


## 最後に - マイナーな改善

わずかな調整で、これを変更して、結果がOpenAIからストリーミングされるようにすることができます。
おなじみのタイプライターアニメーションを備えています

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

発見されたリンク： {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://huggingface.co/chat'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}]}



# Hugging Face Company Overview

## About Us
Hugging Face is an innovative AI community dedicated to building the future of machine learning. We offer a collaborative platform where individuals and organizations can log in to explore, create, and share AI models, datasets, and applications. Our vision is to foster an open environment that accelerates the development and adoption of AI technologies.

## Our Offerings
- **Models**: Access over 1,000,000 curated models, including cutting-edge algorithms for diverse applications.
- **Datasets**: Explore a rich library of over 250,000 datasets essential for training AI models.
- **Spaces**: A collaborative arena where users can deploy and share applications.
- **Enterprise Solutions**: Tailored offerings for teams, featuring enterprise-grade security and dedicated support starting at $20/user/month.

## Corporate Culture
At Hugging Face, we believe in the power of community and collaboration. Our open-source ethos encourages contributions from all corners of the AI landscape, with a commitment to transparency and inclusivity. We actively support over 50,000 organizations, including major companies like Meta, Amazon, and Microsoft, in their AI endeavors.

## Careers
Join our team and be part of a dynamic environment that thrives on innovation. We are seeking passionate individuals who are eager to contribute to the AI community. Explore current openings on our careers page.

## Community Engagement
Hugging Face is not just a platform; it is a vibrant community. We host events, maintain active forums, and offer various resources for learning and collaboration. We invite you to become a part of our journey in redefining what's possible with AI.

## Follow Us
For updates and more information, connect with us on:
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface/)
- [Discord](https://discord.gg/huggingface)

Join us at Hugging Face, where we are making the future of AI accessible and collaborative for everyone!



In [23]:
# HuggingFaceのパンフレットを作成するときに、システムプロンプトをユーモラスなバージョンに変更してみてください。

System_prompt = "あなたは、会社のWebサイトからいくつかの関連ページの内容を分析するアシスタントです\
そして、見込み客、投資家、新兵のために、会社についてのユーモラスで面白い、ジョークのパンフレットを作成します。マークダウンで応答します。\
情報がある場合は、企業文化、顧客、キャリア/ジョブの詳細を含めてください。"

stream_brochure("HuggingFace", "https://huggingface.co")

発見されたリンク： {'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Overview

## Introduction
Hugging Face is at the forefront of the artificial intelligence community, dedicated to building the future through collaboration and innovation. The platform serves as a central hub for machine learning professionals and enthusiasts to share, explore, and collaborate on models, datasets, and applications.

## Company Vision
Hugging Face aims to empower the next generation of machine learning engineers, scientists, and end-users by providing open-source tools and a community-driven space that promotes ethical AI practices. 

## Key Offerings
- **Models**: Access and manage over 1 million models across various machine learning modalities including text, image, video, and audio.
- **Datasets**: Collaborate and utilize over 250,000 datasets for diverse machine learning tasks.
- **Spaces**: Host and run applications that leverage AI technologies, with trending apps making it easy to foster collaboration.
- **Enterprise Solutions**: Dedicated support and tools for organizations to build AI with security and scalability.

## Company Culture
The diverse and dynamic culture at Hugging Face is rooted in community and collaboration. As a platform developed by and for the machine learning community, it values contributions from all stakeholders, including researchers, developers, and organizations. The company nurtures a culture of learning, sharing, and ethical AI practices.

## Customer Base
With over 50,000 organizations using Hugging Face, the platform supports a variety of sectors including tech giants like Google, Amazon, Microsoft, and Meta, as well as non-profit entities. This broad customer base highlights Hugging Face’s versatility and appeal within the industry.

## Careers
Hugging Face provides a thriving environment for professionals passionate about AI and machine learning. The company is always on the lookout for talent across various disciplines, empowering team members with opportunities for growth and collaboration. Positions span technical roles such as software engineers and data scientists to community and product management roles.

## Join Us
Explore your potential at Hugging Face and become part of a community-driven initiative dedicated to shaping the future of AI. To view open positions or learn more about the company, visit our [Careers page](https://huggingface.co/jobs).

## Contact and Resources
- **Website**: [huggingface.co](https://huggingface.co)
- **Social Media**: Follow us on [Twitter](https://twitter.com/huggingface), [GitHub](https://github.com/huggingface), and [LinkedIn](https://linkedin.com/company/huggingface) to stay updated.
- **Documentation & Support**: Access extensive resources and community support through our [Docs](https://huggingface.co/docs) and [Forum](https://huggingface.co/forum).

Together, let’s build an open and ethical AI future!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">ビジネスアプリケーション</h2>
            <span style="color:#181;">この演習では、1日目のコードを拡張して複数のLLM呼び出しを行い、ドキュメントを生成しました。

これはおそらく、LLMSへの複数の呼び出しを組み合わせたため、エージェントAI設計パターンの最初の例です。これにより、2週目にさらに登場し、完全に自律剤ソリューションを構築する際に、8週目にエージェントAIに大きな方法で戻ります。

この方法でコンテンツを生成することは、非常に一般的なユースケースの1つです。要約と同様に、これはあらゆるビジネス垂直に適用できます。マーケティングコンテンツを作成し、スペックから製品チュートリアルを生成し、パーソナライズされた電子メールコンテンツを作成します。コンテンツ生成をビジネスに適用する方法を調べて、自分自身を概念実証プロトタイプにしてみてください。他の学生がコミュニティコントリビューションズフォルダーで行ったことをご覧ください - 非常に多くの貴重なプロジェクト - それはワイルドです！</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">第2週(とても楽しいです)に進む前に</h2>
            <span style="color:#900;">第1週の終わりの課題については、第1週の演習ノートブックをご覧ください。Frontier APIの操作に関する基本的な練習ができ、第2週に向けてしっかりと準備できます。</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">2週目に移動する前に（これはたくさんの楽しみです）</h2>
            <span style="color:#f71;">1. コースのリソースは <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">こちら</a>から入手できます。<br/>
2. 私は LinkedIn に <a href="https://www.linkedin.com/in/eddonner/">こちら</a> から登録しています。コースを受講されている方と交流できるのを楽しみにしています。<br/>
3. Twitter も試していて、<a href="https://x.com/edwarddonner">@edwarddonner<a> でアカウントを作成しています。皆さんからやり方を教えていただければ幸いです。</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">最後に！特別なお願いがあります</h2>
            <span style="color:#090;">
                編集者から、Udemyでこのコースを受講生に評価してもらうと非常に大きな違いが出ると言われました。評価は、Udemyがこのコースを他の受講生に公開するかどうかを決める主な基準の一つだからです。もし評価していただけるなら、本当に嬉しいです！また、何かご不明な点がございましたら、いつでもed@edwarddonner.comまでご連絡ください。
            </span>
        </td>
    </tr>
</table>